In [1]:
## mashup script for batch changes
import os, osgeo
# del os.environ['PROJ_LIB']
import math
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
from geopandas import GeoDataFrame
from pandas import DataFrame
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
from pyproj import proj
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum
import rasterio
from rasterstats import zonal_stats
import xarray as xr
import numpy as np
from osgeo import gdal, gdalconst, osr
import sys, time, ogr
import libpysal
gdal.UseExceptions()  # not required, but a good idea
gdal.AllRegister()
# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

ModuleNotFoundError: No module named 'osgeo'

In [3]:
raster=r"D:\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\02-satellite\Makkah-WV2-ST-2023_I602488\Makkah-WV2-ST-2023-MOS\23JAN19075627-S2AM-Makkah-WV2-ST-2023-tiled.png"
# aoi = "/Users/philipp/BPLA Dropbox/03 Planning/1232-T2-GIS and Remote Sensing-TMO/02_GIS_Data/LandCoverClassificaiton/lcc-trial/aoi-lcc.geojson"
training = r"D:\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\03-poc\mekka-training-tree-0.shp"

In [4]:
# read training polygones
train = gpd.read_file(training) # this may contain multipolygons
train = train.to_crs(32637)
train_ex = train.explode()
train_ex.geometry=train_ex.centroid # we should use the full range of values inside the polygon not jsust the centroid 
# train_ex['lon'] = train_ex.centroid.x  
# train_ex['lat'] = train_ex.centroid.y
# TODO: https://www.matecdev.com/posts/random-points-in-polygon.html

C:\Users\DroneAdmin\AppData\Local\Temp\ipykernel_13640\3821961065.py:4: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  train_ex = train.explode()


In [5]:
with rasterio.open(raster, 'r') as ds:
    arr = ds.read()  # read all raster values
    transform = ds.transform 
    crs = ds.crs
    res=ds.res

print(arr.shape)
print(crs)
print(res)
print(transform)

(3, 123881, 148236)
PROJCS["WGS 84 / UTM zone 37N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",39],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32637"]]
(0.46, 0.46)
| 0.46, 0.00, 552743.82|
| 0.00,-0.46, 2398842.04|
| 0.00, 0.00, 1.00|


In [6]:
# Transform the CRS to WGS84 lat/lon
train_ex = train_ex.to_crs('epsg:32637')

# with gw.open(raster) as src:
#     df = src.gw.extract(train_ex)
with gw.config.update(sensor='bgr'):
    with gw.open(raster) as src:
        df = src.gw.extract(
            train_ex,
            band_names=src.band.values.tolist()
        )

NameError: name 'gw' is not defined